In [ ]:
import serial
import sqlite3
import numpy as np
from collections import deque
import time

ser = serial.Serial('/dev/cu.usbmodem2E79A1B12', 115200)  # Connect to the Arduino serial port

# Parameters for wave detection
window_size = 50  # Number of data points to keep in the sliding window
magnitude_threshold = 1000  # Minimum magnitude of a data point to be considered for zero crossing
zero_crossing_threshold = 4  # Minimum number of zero crossings to detect a wave

# Baseline Statistics
baseline_means = {'gyro_X': 0, 'gyro_Y': 0, 'gyro_Z': 0}  # Mean for each axis
baseline_stds = {'gyro_X': 50, 'gyro_Y': 50, 'gyro_Z': 50} # Standard deviation for each axis
k = 2  # Multiplier for standard deviation to set the threshold

# Dynamic threshold calculation
stationary_thresholds = {
    axis: baseline_means[axis] + k * baseline_stds[axis]    # Threshold for each axis
    for axis in ['gyro_X', 'gyro_Y', 'gyro_Z']
}

# Deques to store recent values for each axis
gyro_x_values = deque(maxlen=window_size)
gyro_y_values = deque(maxlen=window_size)   
gyro_z_values = deque(maxlen=window_size)

currently_waving = False    # Flag to keep track of whether a wave is currently being detected

conn = sqlite3.connect('gyro_data.db')  # Connect to the SQLite database
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS gyroscope_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    gyro_x REAL,
    gyro_y REAL,
    gyro_z REAL
)
''')
conn.commit()

def is_wave_detected(y_data):
    """Function to check if a wave is detected based on the current Y-axis data window."""
    zero_crossings = ((np.diff(np.sign(y_data)) != 0) & (np.abs(y_data[1:]) > magnitude_threshold)).sum()   # Count zero crossings 
    return zero_crossings >= zero_crossing_threshold    # Return True if the number of zero crossings is above the threshold

def is_stationary(x_data, y_data, z_data, thresholds):
    """Function to check if all axes are below the dynamically derived threshold for stopping."""
    return (
        np.max(np.abs(x_data)) < thresholds['gyro_X'] and   # Check if the maximum value of each axis is below the threshold
        np.max(np.abs(y_data)) < thresholds['gyro_Y'] and  
        np.max(np.abs(z_data)) < thresholds['gyro_Z']
    )

try:
    while True:
        if ser.in_waiting > 0:  # Check if there is data in the serial buffer
            line = ser.readline().decode('utf-8', errors='ignore').strip()  # Read the line from the serial buffer

            if 'gyro_X' in line and 'gyro_Y' in line and 'gyro_Z' in line:  # Check if the line contains gyro data
                try:
                    parts = line.split(',') 
                    gyro_x_str = next((s for s in parts if 'gyro_X' in s), None)    # Extract gyro data from the line
                    gyro_y_str = next((s for s in parts if 'gyro_Y' in s), None)
                    gyro_z_str = next((s for s in parts if 'gyro_Z' in s), None)
                    
                    if gyro_x_str and gyro_y_str and gyro_z_str:    # Check if all gyro data is present
                        gyro_x = float(gyro_x_str.split(':')[1])    # Extract the values from the string
                        gyro_y = float(gyro_y_str.split(':')[1])
                        gyro_z = float(gyro_z_str.split(':')[1])

                        gyro_x_values.append(gyro_x)    # Append the values to the deques
                        gyro_y_values.append(gyro_y)
                        gyro_z_values.append(gyro_z)

                        # Insert data into SQLite database
                        cursor.execute('''
                            INSERT INTO gyroscope_data (gyro_x, gyro_y, gyro_z)
                            VALUES (?, ?, ?)
                        ''', (gyro_x, gyro_y, gyro_z))
                        conn.commit()

                        if len(gyro_y_values) == window_size:   # Check if the sliding window is full
                            wave_detected = is_wave_detected(np.array(gyro_y_values))   # Check if a wave is detected

                            if wave_detected and not currently_waving:  # Transition from stationary to waving
                                print("Wave Start Detected!")
                                currently_waving = True # Set the flag to indicate that a wave is being detected

                            elif currently_waving and is_stationary(    # Transition from waving to stationary
                                np.array(gyro_x_values), np.array(gyro_y_values), np.array(gyro_z_values), stationary_thresholds    # Check if all axes are below the threshold
                            ):
                                print("Wave End Detected!")
                                currently_waving = False    # Set the flag to indicate that no wave is being detected

                except ValueError:
                    print("Error parsing data, skipping this entry.")   # Handle errors in parsing data

except KeyboardInterrupt:   # Handle keyboard interrupt to stop the program
    print("Stopped by User")

finally:
    ser.close() # Close the serial connection
    conn.close()    # Close the SQLite connection


In [73]:
import sqlite3
import pandas as pd
import plotly.graph_objs as go
import plotly.subplots as sp

conn = sqlite3.connect('gyro_data.db')  # Connect to the SQLite database
cursor = conn.cursor()

# Query to retrieve all data in gyroscope_data table
query = '''
SELECT * FROM gyroscope_data
WHERE timestamp >= '2024-12-04' AND timestamp < '2024-12-04 14:55:00'
ORDER BY timestamp ASC
'''
rows = cursor.execute(query).fetchall() # Fetch all rows from the query result

df = pd.DataFrame(rows, columns=['id', 'timestamp', 'gyro_x', 'gyro_y', 'gyro_z'])  # Create a DataFrame from the rows

df['timestamp'] = pd.to_datetime(df['timestamp'])   # Convert timestamp column to datetime format for plotting

# Create subplots for Gyro X, Y, and Z Axes
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True,   # Create subplots with shared x-axis
                       subplot_titles=('Gyroscope X Axis Data', 'Gyroscope Y Axis Data', 'Gyroscope Z Axis Data'),  # Subplot titles
                       vertical_spacing=0.1)    # Vertical spacing between subplots

trace_x = go.Scatter(   # Create a scatter plot for Gyro X data
    x=df['timestamp'],
    y=df['gyro_x'],
    mode='lines',
    name='Gyro X',
    line=dict(color='red')
)
fig.add_trace(trace_x, row=1, col=1)

trace_y = go.Scatter(   # Create a scatter plot for Gyro Y data
    x=df['timestamp'],
    y=df['gyro_y'],
    mode='lines',
    name='Gyro Y',
    line=dict(color='green')
)
fig.add_trace(trace_y, row=2, col=1)

trace_z = go.Scatter(   # Create a scatter plot for Gyro Z data
    x=df['timestamp'],
    y=df['gyro_z'],
    mode='lines',
    name='Gyro Z',
    line=dict(color='blue')
)
fig.add_trace(trace_z, row=3, col=1)

fig.update_layout(  # Update layout of the figure with title and axis labels
    height=900,  # Height of the figure
    title_text='Gyroscope Data for X, Y, and Z Axes',
    xaxis=dict(title='Timestamp'),
    yaxis=dict(title='Angular Velocity (degrees/second)'),
    showlegend=False,  # Hide legend for individual traces
)

# Update each subplot axis titles and horizontal line at y=0
fig.update_yaxes(title_text='Angular Velocity (degrees/second)', row=1, col=1)
fig.update_yaxes(title_text='Angular Velocity (degrees/second)', row=2, col=1)
fig.update_yaxes(title_text='Angular Velocity (degrees/second)', row=3, col=1)
fig.update_xaxes(title_text='Timestamp', row=3, col=1)

# Add horizontal lines at y=0 for each subplot
fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=0, y1=0,
              line=dict(color="black", width=0.5), row=1, col=1)
fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=0, y1=0,
              line=dict(color="black", width=0.5), row=2, col=1)
fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=0, y1=0,
              line=dict(color="black", width=0.5), row=3, col=1)

# Add grid lines
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)

fig.show()  # Display the figure with Gyro X, Y, and Z data

print("Statistical Summary of Gyroscope Data (in degrees/second):")
print(df[['gyro_x', 'gyro_y', 'gyro_z']].describe())    # Display statistical summary of Gyro X, Y, and Z data

df.to_csv('gyro_data_analysis.csv', index=False)    # Save the DataFrame to a CSV file

conn.close()    # Close the SQLite connection


Statistical Summary of Gyroscope Data (in degrees/second):
             gyro_x        gyro_y        gyro_z
count   6390.000000   6390.000000   6390.000000
mean      -2.743192    -21.522066     -3.246009
std     1443.784726   1310.573051   3133.998647
min   -31176.000000 -21234.000000 -21408.000000
25%      -48.000000    -59.000000    -40.000000
50%        0.000000     -1.000000      0.000000
75%       42.750000     28.750000     45.000000
max    31162.000000  22807.000000  32767.000000


In [55]:
import serial
import sqlite3
import numpy as np
from collections import deque
import time

ser = serial.Serial('/dev/cu.usbmodem2E79A1B12', 115200)  # # Connect to Arduino via the serial port

magnitude_threshold = 1800  # Minimum magnitude to consider a valid movement.
stationary_threshold = 400  # Maximum magnitude to consider the system as stationary.
stabilization_duration = 2  # Duration to wait for stabilization (in seconds).

window_size = 100  # Number of samples in the sliding window.
gyro_x_values = deque(maxlen=window_size)   # Deque to store X-axis gyroscope values.
gyro_y_values = deque(maxlen=window_size)   # Deque to store Y-axis gyroscope values.
gyro_z_values = deque(maxlen=window_size)   # Deque to store Z-axis gyroscope values.
magnitude_values = deque(maxlen=window_size)    # Deque to store magnitude values.

currently_waving = False    # Flag to track if user is currently waving.
stabilized = False  # Flag to track if system has stabilized.
start_stabilization_time = None   # Variable to store the start time of stabilization.

conn = sqlite3.connect('gyro_data.db')  # Connect to SQLite database.
cursor = conn.cursor()  # Create a cursor object to execute SQL queries.

def calculate_magnitude(x, y, z):
    """Calculate the magnitude of the combined gyroscope vector.
    
    Args:
        x (float): X-axis gyroscope value.
        y (float): Y-axis gyroscope value.
        z (float): Z-axis gyroscope value.
    
    Returns:
        float: Magnitude of the combined gyroscope vector.

    Raises:
        ValueError: If any of the input values are invalid.
    """
    return np.sqrt(x**2 + y**2 + z**2)

try:
    while True:
        if ser.in_waiting > 0:  # Check if there is data in the serial buffer
            line = ser.readline().decode('utf-8', errors='ignore').strip()

            if 'gyro_X' in line and 'gyro_Y' in line and 'gyro_Z' in line:
                try:
                    parts = line.split(',')
                    gyro_x_str = next((s for s in parts if 'gyro_X' in s), None)
                    gyro_y_str = next((s for s in parts if 'gyro_Y' in s), None)
                    gyro_z_str = next((s for s in parts if 'gyro_Z' in s), None)
                    
                    if gyro_x_str and gyro_y_str and gyro_z_str:
                        gyro_x = float(gyro_x_str.split(':')[1])
                        gyro_y = float(gyro_y_str.split(':')[1])
                        gyro_z = float(gyro_z_str.split(':')[1])
                        
                        magnitude = calculate_magnitude(gyro_x, gyro_y, gyro_z) # Calculate the magnitude of the gyroscope vector

                        gyro_x_values.append(gyro_x)
                        gyro_y_values.append(gyro_y)
                        gyro_z_values.append(gyro_z)
                        magnitude_values.append(magnitude)

                        # Insert data into SQLite database
                        cursor.execute('''
                            INSERT INTO gyroscope_data (gyro_x, gyro_y, gyro_z)
                            VALUES (?, ?, ?)
                        ''', (gyro_x, gyro_y, gyro_z))
                        conn.commit()

                        # Stabilisation logic: wait until magnitude stays low for a consistent period
                        if not stabilized:  # Check if the system is still stabilizing
                            if magnitude < stationary_threshold:    # Check if the magnitude is below the stationary threshold
                                if start_stabilization_time is None:    # Start timing when magnitude falls below stationary threshold
                                    start_stabilization_time = time.time()
                                
                                elif time.time() - start_stabilization_time >= stabilization_duration:  # Check if stabilization duration has passed
                                    print("Stabilization Complete. System Ready.")
                                    stabilized = True
                                    start_stabilization_time = None
                            else:
                                start_stabilization_time = None # If magnitude exceeds threshold, reset stabilization timing

                            continue  # Skip wave detection during the stabilization phase

                        # Basic wave detection logic
                        if magnitude > magnitude_threshold and not currently_waving:    # Check if magnitude exceeds threshold and not currently waving
                            print("Wave Start Detected!")
                            currently_waving = True

                        elif magnitude < stationary_threshold and currently_waving: # Check if magnitude falls below threshold and currently waving
                            print("Wave Start Detected!")
                            currently_waving = False

                except ValueError:  # Handle errors in parsing data
                    print("Error parsing data, skipping this entry.")

except KeyboardInterrupt:   # Handle keyboard interrupt to stop the program
    print("Stopped by User")

finally:
    ser.close() # Close the serial connection
    conn.close()    # Close the SQLite connection


Stabilization Complete. System Ready.
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Wave Start Detected!
Wave End Detected!
Stopped by User
